# Segmenting and Clustering Neighborhoods in Toronto: Assignment


#### By Jonathan de Steuben

## Part 1: Building a DataFrame using BeautifulSoup

We'll begin by importing a few packages.

In [1]:
#Importing packages for web-scraping

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

For our first step, we'll scrape the Wikipedia page and populate an empty DataFrame. We need to focus on the HTML dealing with the table we're trying to use.

In [2]:
#Creating URL and extracting data

city_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(city_url, 'lxml')

table = soup.find('table',{'class':'wikitable sortable'})

table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

toronto_df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
toronto_df = toronto_df[~toronto_df['PostalCode'].isnull()]

toronto_df.head(10)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
9,M8A,Not assigned,Not assigned
10,M9A,Queen's Park,Not assigned


Now that we have our table as a DataFrame, we need to clean the data further. First, let's examine data types.

In [3]:
#Examining types in our DataFrame

print(toronto_df.dtypes)

PostalCode      object
Borough         object
Neighborhood    object
dtype: object


Our next step is to remove rows with unassigned boroughs.

In [4]:
#Removing rows with unassigned boroughs

index_names = toronto_df[toronto_df['Borough'] == 'Not assigned'].index

toronto_df.drop(index_names, inplace=True)

toronto_df.reset_index(drop=True, inplace=True)

toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


This looks better. Let's remove the unassigned values from the Neighborhood column.

In [5]:
#Searching the DataFrame of unassigned neighborhoods

missing_n = toronto_df[toronto_df['Neighborhood'] == 'Not assigned'].index

missing_n

Int64Index([6], dtype='int64')

We can see here that there is only 1 missing neighborhood name in the sixth index position. Since we only found one, let's give it the borough value.

In [6]:
#Fixing the last neighborhood

toronto_df['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [7]:
print(toronto_df.shape)
print(toronto_df.count())

(210, 3)
PostalCode      210
Borough         210
Neighborhood    210
dtype: int64


Now that we've cleaned the DataFrame for boroughs and neighborhoods, we'll need to group neighborhoods around postal codes.

In [8]:
#Consolidating neighborhoods around postal codes

postCount = toronto_df['PostalCode'].nunique()
b_Count = toronto_df['Borough'].nunique()
n_Count = toronto_df['Neighborhood'].nunique()

print("# of unique postal codes:", postCount)
print("# of unique boroughs:", b_Count)
print("# of unique neighborhoods:", n_Count)
print("Shape of the current DataFrame:", toronto_df.shape)

# of unique postal codes: 103
# of unique boroughs: 11
# of unique neighborhoods: 207
Shape of the current DataFrame: (210, 3)


We can see that we'll need 103 unique postal codes, 11 unique boroughs, and 207 unique neighborhoods. Let's group our neighborhoods:

In [9]:
#Grouping the postal codes and combining our neighborhoods.

t_df = toronto_df.groupby('PostalCode').agg({'Borough':'first','Neighborhood':', '.join}).reset_index()

t_df.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


After inspecting a few rows, this DataFrame looks clean and ready. Let's take a look at the counts again:

In [10]:
#Counts on our newly cleaned DataFrame

postCount = t_df['PostalCode'].nunique()
b_Count = t_df['Borough'].nunique()
#n_Count = t_df['Neighborhood'].nunique()

print("# of unique postal codes:", postCount)
print("# of unique boroughs:", b_Count)
#print("# of neighborhoods:", n_Count)

print("Shape of the cleaned DataFrame:", t_df.shape)

# of unique postal codes: 103
# of unique boroughs: 11
Shape of the cleaned DataFrame: (103, 3)


We're now ready to move on to the next section.

## Part 2: Appending neighborhood coordinates

Unfortunately, I wasn't able to get the geocoder package to load properly. I spent roughly 12 days browsing Stack and other tutorial sites to no avail. However, data science isn't a one-and-done deal. I plan to revisit this specific step after completing the course so I can get the geocoder package (or another) to work.

This notebook was created using IBM Watson Studio. My plan is to try replicating this project using Anaconda to see if this might fix things.

Until then, we press on:

In [11]:
#Importing the .csv file provided by Coursera to append locations to the t_df dataframe

loca = pd.read_csv('https://cocl.us/Geospatial_data')

loca.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

loca.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


This is the first time we've seen the location dataframe. My first observation was the order doesn't necessarily match up with what we have in t_df, our cleaned dataframe. We're going to join these two next.

In [12]:
#Using concat to combine the two dataframes by column, using PostalCode

t_df_loca = pd.merge(t_df, loca, on='PostalCode')

t_df_loca.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Excellent! This step frustrated me to no end. This was an enormous learning experience because I began to lose hope. Sometimes you just need to keep pushing. This was a much bigger obstacle to overcome than I anticipated.

But still, we press on.

In the final part, we'll put everything together and begin clustering and visualizing our data.

## Part 3: Exploring and clustering neighborhoods

Next, we're going to explore and cluster neighborhoods. I will use API keys from my Foursquare Developer account to gather the data on venues.

In [13]:
#Importing visualization pacakges
import matplotlib.cm as cm
import matplotlib.colors as colors

#Importing JSON related packages
import json
from pandas.io.json import json_normalize

#Importing k-means for clustering
from sklearn.cluster import KMeans

#Installing and importing the folium package
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

Somehow, I was finally able to get this package to load. We're going to use this to find the coordinates for Toronto, then we'll visualize it with Folium.

In [14]:
from geopy.geocoders import Nominatim

In [15]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto, Canada are 43.653963, -79.387207.


With our geographic coordinates now in-hand, we'll visualize Toronto and the neighborhoods:

In [16]:
t_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(t_df_loca['Latitude'], t_df_loca['Longitude'], t_df_loca['Borough'], t_df_loca['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(t_map)
    
t_map

Now I'll use my API key info to start working with the Toronto data and gather nearby venues of interest. I've hidden this cell for obvious reasons.

In [5]:
# The code was removed by Watson Studio for sharing.

In [18]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's use our new function to get data for Toronto:

In [19]:
toronto_venues = getNearbyVenues(names=t_df_loca['Neighborhood'],
                                latitudes=t_df_loca['Latitude'],
                                longitudes=t_df_loca['Longitude']
                                )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(2216, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",12,12,12,12,12,12
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,55,55,55,55,55,55


In [22]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 unique categories.


Let's analyze the data using one hot encoding:

In [23]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00


In [25]:
toronto_grouped.shape

(100, 266)

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1   Steakhouse  0.04
2          Bar  0.04
3         Café  0.04
4   Restaurant  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4   Mediterranean Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                 Playground  0.33
1                Coffee Shop  0.33
2                       Park  0.33
3  Middle Eastern Restaurant  0.00
4                      Motel  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.17
1        Discount Store  0.08
2              Pharmacy  0.08
3  Fast Food Restaurant  0.08
4   Fried Chicken Joint  0.08


----Alderwood, Long Branch----
 

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#Creating a new dataframe        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Bar,Café,Burger Joint,Breakfast Spot,Cosmetics Shop,Thai Restaurant,Restaurant,Bakery
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Coffee Shop,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Liquor Store,Discount Store,Pharmacy,Pizza Place,Sandwich Place,Coffee Shop,Fried Chicken Joint,Japanese Restaurant,Fast Food Restaurant
4,"Alderwood, Long Branch",Pizza Place,Gym,Pub,Coffee Shop,Pharmacy,Skating Rink,Sandwich Place,Pool,Athletics & Sports,Deli / Bodega


This is a satisfying result. Our last step is to cluster this data.

In [29]:
#Setting cluster # to 5
k = 5

toronto_grouped_cluster = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = k, random_state = 0).fit(toronto_grouped_cluster)

kmeans.labels_[0:10]

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [30]:
#Applying cluster labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = t_df_loca

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,History Museum,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Electronics Store,Intersection,Pizza Place,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Dim Sum Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Hakka Restaurant,Drugstore,Donut Shop


Visualizing our newly clustered neighborhoods:

In [31]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius = 5,
    popup = label,
    #color = rainbow[cluster-1],
    fill = True,
    #fill_color = rainbow[cluster-1],
    fill_opacity = 0.7).add_to(map_clusters)
    
map_clusters

This looks good! I encountered issues with the color, so this is another thing I need to make note of when I come back to this project to make edits and improvements.

#### Cluster 1:

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,1.0,Playground,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
14,Scarborough,1.0,Playground,Coffee Shop,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
23,North York,1.0,Convenience Store,Park,Bank,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
25,North York,1.0,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
30,North York,1.0,Park,Playground,Airport,Snack Place,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
31,North York,1.0,Grocery Store,Park,Bank,Hotel,Comfort Food Restaurant,Comic Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
40,East York,1.0,Pizza Place,Coffee Shop,Park,Convenience Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
44,Central Toronto,1.0,Park,Bus Line,Swim School,Construction & Landscaping,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
48,Central Toronto,1.0,Tennis Court,Trail,Park,Playground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
50,Downtown Toronto,1.0,Park,Playground,Trail,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,History Museum,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Department Store
2,Scarborough,2.0,Electronics Store,Intersection,Pizza Place,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Dim Sum Restaurant,Diner,Discount Store
3,Scarborough,2.0,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
4,Scarborough,2.0,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Hakka Restaurant,Drugstore,Donut Shop
6,Scarborough,2.0,Discount Store,Coffee Shop,Department Store,Convenience Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,Scarborough,2.0,Bakery,Bus Line,Park,Soccer Field,Ice Cream Shop,Bus Station,Intersection,Metro Station,Doner Restaurant,Donut Shop
8,Scarborough,2.0,American Restaurant,Motel,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
9,Scarborough,2.0,Café,College Stadium,Skating Rink,General Entertainment,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
10,Scarborough,2.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
11,Scarborough,2.0,Smoke Shop,Breakfast Spot,Bakery,Middle Eastern Restaurant,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Donut Shop


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,3.0,Baseball Field,Women's Store,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
97,North York,3.0,Baseball Field,Women's Store,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,4.0,Cafeteria,Women's Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


Thanks for taking the time to browse my work! After this notebook is graded for the IBM Capstone project, I will be revisiting this to make changes and clean things up a bit.